# Real-time Hand Gesture Recognition using MediaPipe

## 1. Import Required Libraries

Import OpenCV, MediaPipe, NumPy, and other necessary libraries for webcam access and hand detection. 

**Please see the compability between mediapipe and python. The current version of mediapipe only support until python version 3.12.**

In [49]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time
from collections import deque
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")
print(f"OpenCV version: {cv2.__version__}")
print(f"MediaPipe version: {mp.__version__}")

✓ Libraries imported successfully!
OpenCV version: 4.11.0
MediaPipe version: 0.10.21


## 2. MediaPipe Hand Detection Setup

Initialize MediaPipe Hands solution with detection and tracking confidence parameters for optimal performance.

In [50]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Create Hands detector with confidence parameters
hands = mp_hands.Hands(
    static_image_mode=False,  # For real-time video
    max_num_hands=2,  # Detect up to 2 hands
    min_detection_confidence=0.7,  # Minimum confidence for hand detection
    min_tracking_confidence=0.5  # Minimum confidence for hand tracking
)

# Hand landmark names (21 landmarks per hand)
HAND_LANDMARKS = [
    "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",
    "INDEX_MCP", "INDEX_PIP", "INDEX_DIP", "INDEX_TIP",
    "MIDDLE_MCP", "MIDDLE_PIP", "MIDDLE_DIP", "MIDDLE_TIP",
    "RING_MCP", "RING_PIP", "RING_DIP", "RING_TIP",
    "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
]

print(f"✓ MediaPipe Hands initialized with {len(HAND_LANDMARKS)} landmarks per hand")
print(f"Hand landmarks: {HAND_LANDMARKS}")

✓ MediaPipe Hands initialized with 21 landmarks per hand
Hand landmarks: ['WRIST', 'THUMB_CMC', 'THUMB_MCP', 'THUMB_IP', 'THUMB_TIP', 'INDEX_MCP', 'INDEX_PIP', 'INDEX_DIP', 'INDEX_TIP', 'MIDDLE_MCP', 'MIDDLE_PIP', 'MIDDLE_DIP', 'MIDDLE_TIP', 'RING_MCP', 'RING_PIP', 'RING_DIP', 'RING_TIP', 'PINKY_MCP', 'PINKY_PIP', 'PINKY_DIP', 'PINKY_TIP']


I0000 00:00:1765303292.189245 3377403 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2


## 3. Theory of Hand Gesture Recognition

### How MediaPipe Works

MediaPipe detects hands by finding 21 key points (landmarks) on each hand. These landmarks are located at joints and fingertips.

**The 21 Hand Landmarks:**
- 1 wrist point
- 4 points on each finger (thumb, index, middle, ring, pinky)

Each point has (x, y, z) coordinates showing its position.

**How Gestures are Detected:**

We check if fingers are open or closed by comparing distances between landmarks:
- If the tip is far from the base = finger is **open**
- If the tip is close to the base = finger is **closed**

**Examples:**
- **Thumbs Up 👍** = Thumb pointing up, other fingers closed
- **Peace ✌️** = Index + middle fingers up, others closed  
- **Pointing ☝️** = Only index finger up
- **Fist ✊** = All fingers closed
- **Open Hand ✋** = All fingers open


## 4. Utility Functions for Gesture Recognition

Helper functions to calculate distances, angles, and detect finger extensions.

In [ ]:
def distance(p1, p2):
    """Calculate Euclidean distance between two points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def is_finger_extended(mcp, pip, tip):
    """Check if a finger is extended by comparing distances."""
    # If the distance from MCP to TIP is greater than MCP to PIP, finger is extended
    return distance(mcp, tip) > distance(mcp, pip)

def is_hand_open(landmarks):
    """Check if hand is open (all fingers extended)."""
    thumb_extended = is_finger_extended(landmarks[2], landmarks[1], landmarks[4])
    index_extended = is_finger_extended(landmarks[5], landmarks[6], landmarks[8])
    middle_extended = is_finger_extended(landmarks[9], landmarks[10], landmarks[12])
    ring_extended = is_finger_extended(landmarks[13], landmarks[14], landmarks[16])
    pinky_extended = is_finger_extended(landmarks[17], landmarks[18], landmarks[20])
    
    return sum([thumb_extended, index_extended, middle_extended, ring_extended, pinky_extended]) >= 4

def is_fist(landmarks):
    """Check if hand is forming a fist (all fingers closed)."""
    thumb_extended = is_finger_extended(landmarks[2], landmarks[1], landmarks[4])
    index_extended = is_finger_extended(landmarks[5], landmarks[6], landmarks[8])
    middle_extended = is_finger_extended(landmarks[9], landmarks[10], landmarks[12])
    ring_extended = is_finger_extended(landmarks[13], landmarks[14], landmarks[16])
    pinky_extended = is_finger_extended(landmarks[17], landmarks[18], landmarks[20])
    
    return sum([thumb_extended, index_extended, middle_extended, ring_extended, pinky_extended]) <= 1

def is_thumbs_up(landmarks):
    """Check if gesture is thumbs up."""
    # Thumb should be extended and pointing up (high y-coordinate)
    thumb_extended = is_finger_extended(landmarks[2], landmarks[1], landmarks[4])
    # Other fingers closed
    index_extended = is_finger_extended(landmarks[5], landmarks[6], landmarks[8])
    middle_extended = is_finger_extended(landmarks[9], landmarks[10], landmarks[12])
    
    # Thumb tip should be above thumb MCP (lower y value)
    thumb_up = landmarks[4][1] < landmarks[2][1]
    
    return thumb_extended and thumb_up and not index_extended and not middle_extended

def is_peace_sign(landmarks):
    """Check if gesture is peace sign (victory)."""
    # Index and middle fingers extended, others closed
    index_extended = is_finger_extended(landmarks[5], landmarks[6], landmarks[8])
    middle_extended = is_finger_extended(landmarks[9], landmarks[10], landmarks[12])
    ring_extended = is_finger_extended(landmarks[13], landmarks[14], landmarks[16])
    pinky_extended = is_finger_extended(landmarks[17], landmarks[18], landmarks[20])

def is_pointing(landmarks):
    """Check if gesture is pointing (index finger only)."""
    index_extended = is_finger_extended(landmarks[5], landmarks[6], landmarks[8])
    middle_extended = is_finger_extended(landmarks[9], landmarks[10], landmarks[12])
    ring_extended = is_finger_extended(landmarks[13], landmarks[14], landmarks[16])
    pinky_extended = is_finger_extended(landmarks[17], landmarks[18], landmarks[20])
    
    # Only index finger extended
    return index_extended and not middle_extended and not ring_extended and not pinky_extended

def classify_gesture(landmarks):
    """Classify hand gesture from landmarks."""
    if is_thumbs_up(landmarks):
        return "THUMBS UP 👍", 0.9
    elif is_peace_sign(landmarks):
        return "PEACE ✌️", 0.85
    elif is_pointing(landmarks):
        return "POINTING ☝️", 0.8
    elif is_fist(landmarks):
        return "FIST ✊", 0.95
    elif is_hand_open(landmarks):
        return "OPEN HAND ✋", 0.9
    else:
        return "NEUTRAL", 0.5

print("✓ Gesture classification functions defined successfully!")

✓ Gesture classification functions defined successfully!


W0000 00:00:1765303292.200371 3386783 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


W0000 00:00:1765303292.212295 3386783 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## 5. Image Processing Functions

Helper function to process images and detect hand gestures.

In [52]:
import os
from pathlib import Path

# Create output directory for results
output_dir = "gesture_results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Image directory containing hand gesture images
image_dir = "hand_images"  # Change this to your image folder path

# Statistics tracking
detection_stats = {
    'THUMBS UP 👍': 0,
    'PEACE ✌️': 0,
    'POINTING ☝️': 0,
    'FIST ✊': 0,
    'OPEN HAND ✋': 0,
    'NEUTRAL': 0
}

detection_results = []

# Supported image formats
supported_formats = {'.jpg', '.jpeg', '.png', '.bmp'}

print("Hand Gesture Detection from Images")
print("=" * 60)
print(f"Looking for images in: {image_dir}")
print()

# Check if image directory exists
if not os.path.exists(image_dir):
    print(f"⚠️  Image directory '{image_dir}' not found!")
    print(f"Please create the folder and add hand gesture images.")
    print(f"\nExample usage:")
    print(f"1. Create folder: mkdir hand_images")
    print(f"2. Add images: place .jpg or .png files in hand_images/")
    print(f"3. Re-run this cell")
else:
    # Get all image files
    image_files = [f for f in os.listdir(image_dir) 
                   if os.path.isfile(os.path.join(image_dir, f)) 
                   and Path(f).suffix.lower() in supported_formats]
    
    if not image_files:
        print(f"No image files found in '{image_dir}'")
        print(f"Supported formats: {', '.join(supported_formats)}")
    else:
        print(f"Found {len(image_files)} image(s) to process")
        print("-" * 60)
        
        # Process each image
        for idx, img_file in enumerate(image_files, 1):
            img_path = os.path.join(image_dir, img_file)
            
            try:
                # Read image
                image = cv2.imread(img_path)
                
                if image is None:
                    print(f"[{idx}] ✗ Error reading: {img_file}")
                    continue
                
                # Convert BGR to RGB
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                # Process with MediaPipe
                results = hands.process(rgb_image)
                
                # Copy image for annotation
                annotated_image = image.copy()
                h, w, c = annotated_image.shape
                
                # Extract gesture info
                gesture_info = {
                    'filename': img_file,
                    'hands_detected': 0,
                    'gestures': []
                }
                
                if results.multi_hand_landmarks and results.multi_handedness:
                    for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                        # Extract landmarks
                        landmarks = []
                        for lm in hand_landmarks.landmark:
                            landmarks.append((lm.x, lm.y, lm.z))
                        
                        # Classify gesture
                        gesture, confidence = classify_gesture(landmarks)
                        
                        # Store info
                        gesture_info['hands_detected'] += 1
                        gesture_info['gestures'].append({
                            'gesture': gesture,
                            'confidence': confidence,
                            'handedness': handedness.classification[0].label
                        })
                        
                        # Update stats
                        detection_stats[gesture] += 1
                        
                        # Draw landmarks
                        mp_drawing.draw_landmarks(
                            annotated_image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style()
                        )
                        
                        # Add text annotations
                        y_pos = 50 + gesture_info['hands_detected'] * 80
                        cv2.putText(annotated_image, 
                                  f"Hand {gesture_info['hands_detected']} ({handedness.classification[0].label})", 
                                  (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        cv2.putText(annotated_image, 
                                  f"Gesture: {gesture}", 
                                  (10, y_pos + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                        cv2.putText(annotated_image, 
                                  f"Confidence: {confidence:.2%}", 
                                  (10, y_pos + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)
                
                detection_results.append(gesture_info)
                
                # Save annotated image
                output_path = os.path.join(output_dir, f"detected_{img_file}")
                cv2.imwrite(output_path, annotated_image)
                
                # Print result
                print(f"[{idx}] ✓ {img_file}")
                if gesture_info['hands_detected'] > 0:
                    for g_info in gesture_info['gestures']:
                        print(f"     → {g_info['gesture']} ({g_info['confidence']:.0%} confidence)")
                else:
                    print(f"     → No hands detected")
                
            except Exception as e:
                print(f"[{idx}] ✗ Error processing {img_file}: {str(e)}")
        
        print("\n" + "=" * 60)
        print(f"✓ Processing complete! {len(image_files)} image(s) processed")
        print(f"✓ Results saved to: {output_dir}/")


Hand Gesture Detection from Images
Looking for images in: hand_images

Found 4 image(s) to process
------------------------------------------------------------
[1] ✓ peace_sign_2.jpg
     → OPEN HAND ✋ (90% confidence)
[2] ✓ thumbs_up.jpg
     → THUMBS UP 👍 (90% confidence)
     → OPEN HAND ✋ (90% confidence)
[3] ✓ open_hand.jpg
     → OPEN HAND ✋ (90% confidence)
[4] ✓ peace_sign.jpg
     → OPEN HAND ✋ (90% confidence)

✓ Processing complete! 4 image(s) processed
✓ Results saved to: gesture_results/
[4] ✓ peace_sign.jpg
     → OPEN HAND ✋ (90% confidence)

✓ Processing complete! 4 image(s) processed
✓ Results saved to: gesture_results/


## 6. Test Output and Results Analysis

Display detection statistics, visualize results, and analyze performance metrics.

In [53]:
# Print detection summary
print("\n" + "="*60)
print("DETECTION RESULTS")
print("="*60)

if detection_results:
    print(f"Images processed: {len(detection_results)}")
    total_hands = sum(r['hands_detected'] for r in detection_results)
    print(f"Total hands detected: {total_hands}")
    print()
    
    for result in detection_results:
        print(f"📄 {result['filename']}")
        if result['gestures']:
            for g in result['gestures']:
                print(f"   → {g['gesture']} ({g['confidence']:.0%})")
        else:
            print(f"   → No hands detected")
else:
    print("No results yet. Process images first!")



DETECTION RESULTS
Images processed: 4
Total hands detected: 5

📄 peace_sign_2.jpg
   → OPEN HAND ✋ (90%)
📄 thumbs_up.jpg
   → THUMBS UP 👍 (90%)
   → OPEN HAND ✋ (90%)
📄 open_hand.jpg
   → OPEN HAND ✋ (90%)
📄 peace_sign.jpg
   → OPEN HAND ✋ (90%)


## 7. Display Detection Results

View the annotated images with detected hand gestures and landmarks.

In [54]:
# Display detection results status
if os.path.exists(output_dir):
    result_files = [f for f in os.listdir(output_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if result_files:
        print(f"\n✓ Detection results saved! ({len(result_files)} images)")
        print("=" * 60)
        for result_file in sorted(result_files):
            print(f"  → {result_file}")
        print("=" * 60)
        print(f"Annotated images are in: {output_dir}/")
    else:
        print(f"No processed images found in '{output_dir}/'")
else:
    print(f"Results directory '{output_dir}' not found")


✓ Detection results saved! (4 images)
  → detected_open_hand.jpg
  → detected_peace_sign.jpg
  → detected_peace_sign_2.jpg
  → detected_thumbs_up.jpg
Annotated images are in: gesture_results/


# Note about peace sign detection

Peace sign images detected as OPEN HAND. This happens because the ring and pinky fingers are also extended in those images, so the detector classifies it as OPEN HAND (5 fingers extended) instead of PEACE sign. For accurate peace sign detection, the ring and pinky fingers need to be more clearly closed.